In [1]:
from utils import css_from_file
css_from_file('style/style.css')

Data preprocessing
----------------------

Data preprocessing is the most important step in the model preparation. It takes 90% of the time to prepare and clean the data so it can be processed by a predictive algorithm.

Here we have the data from Rossmann competition https://www.kaggle.com/c/rossmann-store-sales.

It is a good example of a dataset with many different types of data.

In [2]:
import pandas as pd 
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [3]:
training_data = pd.read_csv("data/rossmann/train.csv")
store_data = pd.read_csv("data/rossmann/store.csv")

There are information about the Sales (our target).

In [4]:
print("Training data shape", training_data.shape)
training_data.head()

Training data shape (1017209, 9)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


And the stores themselves

In [5]:
print("Store data shape", store_data.shape)
store_data.head()

Store data shape (1115, 10)


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


Let's join the data

In [6]:
combined_data = pd.merge(training_data, store_data, on="Store")

# sample the data
#combined_data = combined_data.sample(frac=0.1).reset_index()

print("Combined data shape", combined_data.shape)
combined_data.head()

Combined data shape (1017209, 18)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


Exercise
----------------------

1. Identify types of data present in the dataset:
    - what would you do with each type of data?
    - are there missing values?
2. Write transformer `PandasSelector` which can select subsets of columns from the dataset.
3. Write transformers for each type of data that convert selected columns to numerical values.
4. Combine it all into 1 pipeline using `make_pipeline` and `make_union` functions.

Hint: you will need those imports
```python
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
```

Explanation:
`BaseEstimator` and `TransformerMixin` are the classes from which you need to inherit in the your transformer class. They are needed for proper pipeline serialization (saving).

`DictVectorizer` is a transformer that can create a matrix from a dictionary of values - it is helpful to convert categorical variables. 

For example:
Let's say you have 2 columns which you want to convert to a matrix: `StoreType` and `Assortment`

In [16]:
# let's convert 2 columns to a list of dictionaries
data_as_dict = list(store_data.ix[:, ["StoreType","Assortment"]].T.to_dict().values())[:10]
data_as_dict

[{'Assortment': 'a', 'StoreType': 'c'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'c', 'StoreType': 'c'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'c', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'},
 {'Assortment': 'c', 'StoreType': 'a'},
 {'Assortment': 'a', 'StoreType': 'a'}]

In [17]:
from sklearn.feature_extraction import DictVectorizer
categorical_transformer = DictVectorizer()
print(categorical_transformer.fit_transform(data_as_dict).todense()) # by default DictVectorizer returns sparse matrix

[[ 1.  0.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  1.  0.]
 [ 1.  0.  1.  0.]]


In [18]:
data_as_dict[0]["Assortment"] = 'b'
print(categorical_transformer.transform(data_as_dict).todense()) # by default DictVectorizer returns sparse matrix

[[ 0.  0.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  1.  0.]
 [ 1.  0.  1.  0.]
 [ 0.  1.  1.  0.]
 [ 1.  0.  1.  0.]]


In [20]:
data_as_dict[0]["Assortment"] = 'b'
data_as_dict[0]["StoreType"] = 'b'
print(categorical_transformer.fit_transform(data_as_dict).todense()) # by default DictVectorizer returns sparse matrix

[[ 0.  1.  0.  0.  1.  0.]
 [ 1.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.  1.]
 [ 1.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.]]


Exercise template
-----------------

Your final process should like like this:
    
```python
from sklearn.preprocessing import Imputer

processing_pipeline = make_pipeline(
    # fill missing values
    Imputer(),
    
    # combine features
    make_union(
        make_pipeline(
            # select categorical data
            # do something with categorical data
        ),
        make_pipeline(
            # select date
            # do something with dates
            # first convert text to date using pd.to_datetime
            # use .dt attribute of pandas column
        ),
        make_pipeline(
            # select numerical data
            # do something with numerical data
        ),
        make_pipeline(
            # make some feature engineering
        )
    )
)
```

In [22]:
combined_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,1,4,2015-07-30,5020,546,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
2,1,3,2015-07-29,4782,523,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,1,2,2015-07-28,5011,560,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
4,1,1,2015-07-27,6102,612,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN


In [74]:
combined_data.dtypes

Store                          int64
DayOfWeek                      int64
Date                          object
Sales                          int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                  object
SchoolHoliday                  int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [67]:
combined_data.Year = pd.DatetimeIndex(combined_data.Date).year
combined_data.Month = pd.DatetimeIndex(combined_data.Date).month
combined_data.Day = pd.DatetimeIndex(combined_data.Date).day
combined_data.Year
np.vstack([combined_data.Year, combined_data.Month, combined_data.Day]).T


array([[2015,    7,   31],
       [2015,    7,   30],
       [2015,    7,   29],
       ..., 
       [2013,    1,    3],
       [2013,    1,    2],
       [2013,    1,    1]], dtype=int32)

In [72]:
combined_data.Store.value_counts(dropna=False)

1023    942
666     942
675     942
163     942
674     942
162     942
673     942
161     942
672     942
160     942
158     942
157     942
668     942
156     942
667     942
154     942
657     942
665     942
153     942
664     942
152     942
663     942
151     942
662     942
150     942
661     942
148     942
659     942
147     942
658     942
       ... 
636     758
637     758
879     758
638     758
129     758
604     758
365     758
132     758
644     758
646     758
136     758
137     758
629     758
115     758
113     758
890     758
893     758
620     758
108     758
619     758
385     758
612     758
100     758
611     758
99      758
900     758
902     758
903     758
904     758
512     758
Name: Store, dtype: int64

In [70]:
combined_data.Promo2SinceWeek.value_counts(dropna = False)

NaN      508031
 14.0     72990
 40.0     62598
 31.0     39976
 10.0     38828
 5.0      35818
 37.0     32786
 1.0      32418
 13.0     29820
 45.0     29268
 22.0     28694
 18.0     27318
 35.0     22814
 9.0      12452
 27.0      9626
 36.0      9236
 48.0      8294
 39.0      4732
 23.0      4342
 44.0      2642
 26.0       942
 6.0        942
 28.0       942
 50.0       942
 49.0       758
Name: Promo2SinceWeek, dtype: int64

In [81]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import Imputer


class PandasSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x.ix[:,self.columns]
    
class CategoricalTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x.T.to_dict().values()
    
class DateTransformer(BaseEstimator, TransformerMixin):
    
    def __init__(self):
        self
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        Year = pd.DatetimeIndex(x).year
        Month = pd.DatetimeIndex(x).month
        Day = pd.DatetimeIndex(x).day
        
        return np.vstack([Year, Month, Day]).T
    
    
from sklearn.preprocessing import Imputer

processing_pipeline = make_pipeline(
    PandasSelector(['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']),
    # fill missing values
    #Imputer(strategy=“mean”),

    # combine features
    make_union(
        make_pipeline(
            PandasSelector(["StoreType", "Assortment"]),
            CategoricalTransformer(),
            DictVectorizer(sparse=False)
            # do something with categorical data
        ),
        make_pipeline(
            # select date
            # do something with dates
            # first convert text to date using pd.to_datetime
            # use .dt attribute of pandas column
            PandasSelector(["Date"]),
            DateTransformer()
        ),
        make_pipeline(
            # select numerical data
            # do something with numerical data
            PandasSelector(['Store', 'DayOfWeek', 'Sales', "Customers", "Open", "Promo", "SchoolHoliday", "CompetitionDistance", 
                            "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", 
                            "Promo2", "Promo2SinceWeek", "Promo2SinceYear"]),
            Imputer(strategy='mean')
        ),
       # make_pipeline(
            # make some feature engineering
       # )
    )
)
    

        
        

# put your solution here

**Double click to see the solution**

<div class='spoiler'>

class PandasSelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x.ix[:,self.columns]
    
    
class PandasToDict(BaseEstimator, TransformerMixin):

    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        return x.T.to_dict().values()

    
class ExtractDateAttributes(BaseEstimator, TransformerMixin):
    
    def __init__(self, date_format=None,
                 attributes=["year","month","day","weekday"]):
        self.date_format = date_format
        self.attributes = attributes
        
    def fit(self, x, y = None):
        return self
    
    def transform(self, x):
        assert x.shape[1] == 1, "This transformer can handle 1 date"
        
        # convert data to date
        dt = pd.to_datetime(x.ix[:,0])
        
        # create an empty DataFrame
        df = pd.DataFrame()
        
        for attr in self.attributes:
            df[attr] = getattr(dt.dt, attr)
            
        return df
    

processing_pipeline = make_pipeline(
    # Select used variables
    PandasSelector(["Open", "Promo", "SchoolHoliday", 
                    "Date", "StoreType", "Assortment",
                    "CompetitionDistance", "CompetitionOpenSinceMonth",
                    "CompetitionOpenSinceYear", "Promo2",
                    "Promo2SinceWeek", "Promo2SinceYear"]),
    
    # combine features
    make_union(
        make_pipeline(
            PandasSelector(["Open", "Promo", "SchoolHoliday", "CompetitionDistance", 
                            "CompetitionOpenSinceMonth", "CompetitionOpenSinceYear", 
                            "Promo2", "Promo2SinceWeek", "Promo2SinceYear"]),
            Imputer(strategy='mean')
        ),
        make_pipeline(
            PandasSelector(["Date"]),
            ExtractDateAttributes()
        ),
        make_pipeline(
            PandasSelector(["StoreType", "Assortment"]),
            PandasToDict(),
            DictVectorizer(sparse=False)
        )
    )
)

</div>

To combine together your data processing pipeline and predictive algorithm you can chain them using `make_pipeline` function.

## Exercise

1. **Run and understand the code below**
   - explain each line <br/><br/>
   
2. **Why the CV is so low?**
   - the problem is to predict future prices is cross validation a good method to check if models learns well? <br/><br/>
   
3. **Use different splitting scheme and compare results** 
   - train on dates < X and test on dates > X, where X is some date
   
4. **Plot the errors of the predictions depending on how far ahead you make the prediction**

In [82]:
from cross_validation import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from metrics import rmspe

est = RandomForestRegressor(verbose=True, n_jobs=-1)

pred = cross_val_predict(make_pipeline(processing_pipeline, est), 
                         combined_data, 
                         np.log1p(combined_data.Sales),
                         cv=5)

print(rmspe(combined_data.Sales, np.expm1(pred)))

ValueError: Unknown string format

Click here to see the solution

<div class="spoiler">

from cross_validation import cross_val_predict
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from metrics import rmspe

est = RandomForestRegressor(verbose=True, n_jobs=-1)
training_data = combined_data[combined_data.Date <= '2015-05-30']
test_data = combined_data[combined_data.Date > '2015-05-30']

model = make_pipeline(processing_pipeline, est)

model.fit(training_data, np.log1p(training_data.Sales))

predictions = model.predict(test_data)

print(rmspe(combined_data.Sales, np.expm1(pred)))

</div>